# Finding Conan
We want to use Chatgpt4 to retrive information out of our scraped data - Polieimeldungen https://www.berlin.de/polizei/polizeimeldungen/

We want the following information:
1. When did the crime happened
2. What crime happened
3. Where did it happened


In [2]:
# Install in terminal
# pip install openai
# pip install --upgrade openai
# npm install openai

# **Import the necessary packages**

In [245]:
import os
import openai
import pandas as pd
from IPython.display import display, Markdown
import requests
import json
import shutil 
import time
import glob
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib as plt
from datetime import datetime

# **Set API_Key secret**

In [318]:
# Get the path where the credential is
path_cred = os.path.join(os.path.dirname(os.path.dirname(os.getcwd())), 'credentials')

# open the credential
with open(os.path.join(path_cred,'secrets.json')) as secrets_file:
    secrets = json.load(secrets_file)
    api_key = secrets['mlh_api_key']

# **select a model**

https://platform.openai.com/docs/api-reference/models


# **connect to API**

# load single json file

In [247]:
path_to_json_files = '/Users/ellenlee/code/hclpush/finding-conan/raw-data/unstructured-data' # TODO
specification_1 = 'categorized-test-latest-2000-files'
specification_2 = 'one-case'
file_name = 'pressemitteilung.1207392.json'
f = open(os.path.join(path_to_json_files, specification_1, specification_2, file_name))
case = json.load(f)

del case['subtitle']

In [248]:
case

{'title': 'Vierjährige von Auto angefahren',
 'place': 'Charlottenburg-Wilmersdorf',
 'date': '17.05.2022',
 'data': [{'number': '1047',
   'description': 'Gestern Nachmittag wurde eine Vierjährige bei einem Verkehrsunfall im Ortsteil Grunewald verletzt. Nach derzeitigen Erkenntnissen und Zeugenaussagen wollte das Kind gegen 16.50 Uhr die Gehwegseite der Auerbachstraße wechseln, löste sich dafür von der Hand seiner Begleiterin, rannte auf die Straße und wurde dort von einem Auto angefahren. In Folge des Zusammenstoßes erlitt das Mädchen Verletzungen am Knie und im Bereich der Rippen, mit denen es vorsorglich in einem Krankenhaus verblieb. Da der 67-jährige Fahrzeugführer während der Sachverhaltsaufnahme geistig abwesend wirkte und nur schwer den Anweisungen der Einsatzkräfte folgen konnte, wurde ihm nach rechtlicher Belehrung ein Atemalkoholtest angeboten. Diesem stimmte er zu. Das Ergebnis: Rund ein Promille. Somit wurde sein Führerschein beschlagnahmt, sein Wagen geparkt und verschlo

In [249]:
path_to_json_files

'/Users/ellenlee/code/hclpush/finding-conan/raw-data/unstructured-data'

# Create Important Functions

In [443]:
def chatGPT(text, api_key):
    url = "https://api.openai.com/v1/completions"
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {api_key}",
    }
    prompt = { 
        "model":"text-davinci-003",
        "prompt":
        """
        You are crime analyzing assistant. The assistant is helpful, clever, and gives accurate answers.

        You define types of crimes according to the following categories:
        Homicide = The act of unlawfully causing the death of another person.
        Hate Crime - Disability = Anti-Mental Disability, Anti-Physical Disability
        Hate Crime - Gender = Anti-Male, Anti-Female
        Hate Crime - Gender Identity = Anti-Transgender, Anti-Gender Non-Conforming
        Hate Crime - Religious = Anti-Buddhist, Anti-Catholic, Anti-Eastern Orthodox (Russian, Greek, Other), Anti-Hindu, Anti-Islamic, Anti-Jehovah’s Witness, Anti-Jewish, Anti-Mormon, Anti-Multiple Religions, Group, Anti-Other Christian, Anti-Other Religion, Anti-Protestant, Anti-Sikh, Anti-Atheism/Agnosticism, etc.
        Hate Crime - Sexual orientation = Anti-Bisexual, Anti-Gay (Male), Anti-Heterosexual, Anti-Lesbian, Anti-Lesbian, Gay, Bisexual, or Transgender (Mixed Group)
        Hate Crime - Racial/Ethnicity = Anti-American Indian or Alaska Native, Anti-Arab, Anti-Asian, Anti-Black or African American, Anti-Hispanic or Latino, Anti-Multiple Races, Group, Anti-Native Hawaiian or Other Pacific Islander, Anti-Other Race/Ethnicity/Ancestry, Anti-White
        Property Crimes = Offenses that involve unlawful interference with someone else's property, such as theft, burglary, or vandalism.
        Verbal Abuse = The use of offensive, derogatory, or threatening language with the intention to harm or intimidate another person verbally.
        Property Damage = Is damage or destruction of real or tangible personal property, 
        Drug Offenses = Violations of laws related to the possession, distribution, manufacturing, or use of illegal substances or controlled substances without proper authorization.
        General Assault = Physical or verbal attacks on another person that result in causing harm or fear.
        Sexual Assault = Non-consensual sexual acts or behavior that involve physical or psychological harm or coercion
        Sexual Harassment = Unwanted sexual advances, comments, or conduct that creates a hostile or uncomfortable environment for the victim
        Unclassified = Cases or incidents that do not fit into any specific predefined category.

        Read the following text of a crime that happened in Berlin, Germany. Give us information about the crime in the form of a key value pair. List the type of crime described in the text according to your definition. If multiple crimes occurred add all types of crimes that are described in the text. List the location as accurate as possible.
        Also list the year, date, and time of the crime. List the sex of the victims and the sex of the offender. List how many victims and how many offenders were involved.  If have no clear answer you response with "unknown".

        Here is one example for the following crime and the desired output:

        Junge Männer mit Messer verletzt
        Polizeimeldung vom 13.05.2023
        Mitte
        Nr. 0749
        Gestern Abend zeigten ein Jugendlicher und zwei junge Männer in der Alex-Wache in Mitte Körperverletzungen, begangen aus einer Gruppe, an. Die beiden 20-Jährigen und ihr 17 Jahre alter Begleiter gaben an, dass sie gegen 21.30 Uhr auf dem Alexanderplatz in Richtung S-Bahnhof unterwegs waren, als sie plötzlich aus einer Gruppe von sechs bis acht Männern heraus zunächst angepöbelt und dann körperlich attackiert wurden. Dabei erlitt einer der 20-Jährigen eine stark blutende Schnittverletzung im Gesicht und sein gleichaltriger Bekannter eine oberflächliche Stichverletzung am Rücken. Der Jugendliche blieb unverletzt. Alarmierte Rettungskräfte brachten die beiden Verletzten in Krankenhäuser. Der am Rücken Verletzte wurde ambulant behandelt, der im Gesicht Verletzte stationär aufgenommen. Einsatzkräfte stellten kurz nach der Anzeigenerstattung in Höhe der Sankt Marienkirche zwei 19-jährige Männer fest, die nach Angaben des Trios zu der angreifenden Gruppe gehörten. Die Kräfte nahmen die beiden Tatverdächtigen fest und führten sie einer erkennungsdienstlichen Behandlung zu. Nach den polizeilichen Maßnahmen wurden beide entlassen. Die Ermittlungen zum Tatgeschehen und zu den anderen Mitgliedern der Gruppe dauern an und wurden von einem Fachkommissariat der Polizeidirektion 5 (City) übernommen.
        Year: 2023
        Date: 13.05
        Time:  21.30 Uhr 
        Type of Crime: General Assault, Verbal Abuse
        Location: Alexanderplatz 
        Victim's Sex: Male
        Offender's Sex: Male
        Number of Victims: 3
        Number of Offenders: 6-8

        Please do this for the following crime:
        """ + str(text),
        "temperature":0.22,
        "max_tokens":1788,
        "top_p":0.46,
        "frequency_penalty":0,
        "presence_penalty":0
    }
    response = requests.post(url, headers=headers, json=prompt)
#     print('response: ')
#     print(response)
    json_response = response.json()  #TODO
    print(f'======== Response.json(): {json_response} ========')
    try:
        text_output = handle_response(json_response)
    except: 
        print("!!!!======== handle_response broke =======!!!!")
    
    return text_output

In [444]:
def handle_response(json_response):
    print('Inside try_api!!')
    num_retry = 0
    num_retry_limit = 5
    
    if "error" in json_response:
        while num_retry < num_retry_limit:
            print(f"!!!!======== Server error occured. Retrying {num_retry+1}th time in 5 seconds... =======!!!!")
            time.sleep(5)  # Wait for 5 seconds
            print(f"!!!!======== Retrying {num_retry+1}th time now... =======!!!!")
            new_response = requests.post(url, headers=headers, json=prompt)
            new_json_response = new_response.json()
            if "error" in new_json_response: 
                num_retry += 1
            elif 'choices' in new_json_response:
                text_output = new_json_response['choices'][0]['text']
                break
            else:
                print("!!!!======== Unexpected error occured in retry_api function! =======!!!!")
                break
    else:
        text_output = json_response['choices'][0]['text']
        
    return text_output

In [445]:
def label_cases_with_gpt(num_case, files_gen, output_dict):
    """
    Calls openai api to label the raw scraped JSON files on criminal cases
    Outputs a dictionary with keys being an unique identifier (combination of index and original case number) 
        and values being the listed case info
    """
    for index, file_path in enumerate(files_gen[:num_case]): # DEBUG
        data = open(file_path)# Get labeling from GPT API
        case = json.load(data)
        del case['subtitle'] # This does not include relevant info
        
        file_name = file_path.split('/')[-1]
        case_id = str(file_name[-12:-5])
        print(f'======== Working on labeling case number {index+1} with GPT: {case_id} ========')
        
        try: 
            raw_gpt_output = chatGPT(case, api_key)
            output_dict[case_id] = raw_gpt_output

        #Move file to folder
            destination_dir_path = os.path.join(path_to_json_files, 'info-extracted-cases')
            if not os.path.exists(destination_dir_path):
                os.makedirs(destination_dir_path)
            shutil.move(file_path, os.path.join(destination_dir_path, file_name))
        except:
            print(f'!!!!!======== Errors occurs while labeling case number {index+1} with case id {case_id} ========!!!!!')
            break
        
    return output_dict

In [446]:
def inject_cases_info_to_dict(input_dict):
    """
    Creates a DataFrame from the dictionary which is the output of openai api labeling
    
    """
    output_dict = {
        'unique_case_id':[],
        'official_case_id':[],
        'Type of Crime':[],
        'Location':[],
        'Year':[],
        'Date':[],  # Modified
        'Time':[],
        "Victim's Sex":[],
        "Offender's Sex":[],
        'Number of Victims':[],
        'Number of Offenders':[]
}   
    for index, (case_id, case_info) in enumerate(input_dict.items()):
        print(f'======== Extracting info from labeled case number {index+1}: {case_id} ========')
        try:
        #Put infos in a dictionary
#         print(case_info)
            gpt_output_lst = case_info.strip().split('\n')
            gpt_output_lst = [ele.strip('\n').strip() for ele in gpt_output_lst]
            print('gpt_output_lst')
            print(gpt_output_lst)
            output_dict['official_case_id'].append(case_id.zfill(7))
            output_dict['unique_case_id'].append(str(index+1).zfill(6) + '_' + case_id)
            for element in gpt_output_lst:
                key, value = element.split(":")
                if key in output_dict.keys():
                    output_dict[key].append(value.strip())

        
        except:
            print(f'!!======== Errors occurs while procesing case id {case_id} ========!!')
            # Move file to folder
            file_name = f'pressemitteilung.{case_id}.json'
            current_file_path = os.path.join(path_to_json_files, 'info-extracted-cases', file_name)
            destination_dir_path = os.path.join(path_to_json_files, 'unsuccessful-cases')
            
            if not os.path.exists(destination_dir_path):
                os.makedirs(destination_dir_path)
            
#             shutil.move(current_file_path, os.path.join(destination_dir_path, file_name))
            break
    
    output_dict_renamed = {key.lower().replace(' ', '_').replace("'s", ""): value 
                               for key, value in output_dict.items()}
    
    return output_dict_renamed

In [447]:
def output_csv(path_to_json_files, df):
    output_folder = 'csv-output'
    output_path = os.path.join(path_to_json_files, output_folder)
    cur_datetime = datetime.now().strftime("%Y-%m-%d %H-%M")
    if not os.path.exists(output_path):
            os.makedirs(output_path)
    
    csv_file_name = f'labeled_{df.shape[0]}_cases_{cur_datetime}.csv'
    df.to_csv(os.path.join(output_path, csv_file_name), sep=',', index=False)
    print(f'file save with name: {csv_file_name}')

In [448]:
def main_process(specification_1, specification_2, path_to_json_files, num_case=5, export_csv=False):
    # Input files
    files_gen = sorted(
            glob.glob(
            os.path.join(path_to_json_files, specification_1, specification_2, 'pressemitteilung*')),
            reverse=True)
    
    
    # Main processing
    
    start_time = time.time()  # Measure run time of GPT labeling
    original_dict = {}
    original_dict_filled = label_cases_with_gpt(num_case, files_gen, original_dict)
    end_time = time.time() # Measure run time of GPT labeling
    runtime_minutes = (end_time - start_time) / 60
    print(f"======= GPT labeling runtime: {runtime_minutes:.2f} minutes======")
    info_dict = inject_cases_info_to_dict(original_dict_filled)
    
    df = pd.DataFrame.from_dict(info_dict)
#     print(df)
    
    if export_csv:
        output_csv(path_to_json_files, df)
        
    return df

In [449]:
# def move_file(file_name, path_to_json_files, current_dir, destination_dir):
#         destination_dir_path = os.path.join(path_to_json_files, destination_dir) #'info-extracted-cases')
#         if not os.path.exists(destination_dir_path):
#             os.makedirs(destination_dir_path)
#         shutil.move(file_path, os.path.join(destination_dir_path, file_name))

# Start looping

### One function for all

In [266]:
df_final = main_process('categorized-test-latest-2000-files', 'one-case', path_to_json_files, 20)

======== Working on labeling case number 1 with GPT: 1290332 ========
======== Response.json(): {'id': 'cmpl-7MsxWzFVEZLDdupJHpqUO1fo363Gm', 'object': 'text_completion', 'created': 1685688834, 'model': 'text-davinci-003', 'choices': [{'text': "\n\nYear: 2023\nDate: 01.02\nTime: 3.30 Uhr\nType of Crime: Homicide\nLocation: Straße Zum Langen See, Treptow-Köpenick\nVictim's Sex: Male\nOffender's Sex: Unknown\nNumber of Victims: 1\nNumber of Offenders: Unknown", 'index': 0, 'logprobs': None, 'finish_reason': 'stop'}], 'usage': {'prompt_tokens': 1553, 'completion_tokens': 74, 'total_tokens': 1627}} ========
======== Working on labeling case number 2 with GPT: 1290192 ========
======== Response.json(): {'id': 'cmpl-7MsxesTyGRW6EyjbMxd29Mffm8bB2', 'object': 'text_completion', 'created': 1685688842, 'model': 'text-davinci-003', 'choices': [{'text': "\n\nYear: 2022\nDate: 20.03\nTime: 22.40 Uhr\nType of Crime: General Assault, Verbal Abuse\nLocation: Mitte, Fernsehturm\nVictim's Sex: Male\nOffe

======== Response.json(): {'id': 'cmpl-7MszGI2r7yxF8h0rJOGMzvJVqXgnv', 'object': 'text_completion', 'created': 1685688942, 'model': 'text-davinci-003', 'choices': [{'text': "\n\nYear: 2023\nDate: 28.01\nTime: 12.15 Uhr\nType of Crime: Homicide, Property Damage\nLocation: Schierker Straße, Neukölln\nVictim's Sex: Female\nOffender's Sex: Male\nNumber of Victims: 1\nNumber of Offenders: 1", 'index': 0, 'logprobs': None, 'finish_reason': 'stop'}], 'usage': {'prompt_tokens': 1557, 'completion_tokens': 72, 'total_tokens': 1629}} ========
======== Working on labeling case number 16 with GPT: 1288953 ========
======== Response.json(): {'id': 'cmpl-7MszP3tdxfzW9lkunRC3A9WhsiYI6', 'object': 'text_completion', 'created': 1685688951, 'model': 'text-davinci-003', 'choices': [{'text': "\n\nYear: 2023\nDate: 28.01\nTime: 3.10 Uhr\nType of Crime: Property Damage, Arson\nLocation: Tempelhof-Schöneberg\nVictim's Sex: Unknown\nOffender's Sex: Unknown\nNumber of Victims: 0\nNumber of Offenders: Unknown", 

ValueError: All arrays must be of the same length

In [267]:
df_final

,unique_case_id,official_case_id,type_of_crime,location,year,date,time,victim_sex,offender_sex,number_of_victims,number_of_offenders


In [ ]:
# Save to CSV
output_csv(path_to_json_files, df_final)

### Funciton seperated

In [450]:
# Get the current files in the specified folder
files_gen = sorted(
        glob.glob(
        os.path.join(path_to_json_files, specification_1, specification_2, 'pressemitteilung*')),
        reverse=True)

In [451]:
original_dict = {}
original_dict_filled = label_cases_with_gpt(20, files_gen, original_dict) #at 1264683

======== Working on labeling case number 1 with GPT: 1264683 ========
======== Response.json(): {'id': 'cmpl-7NIdXCWF5B6oGfi6KShEItAP7KQos', 'object': 'text_completion', 'created': 1685787539, 'model': 'text-davinci-003', 'choices': [{'text': "\n\nYear: 2022\nDate: 15.11\nTime: 11.45 Uhr\nType of Crime: Property Damage, General Assault\nLocation: Cantianstraße, Schönhauser Allee, Prenzlauer Berg, Pankow\nVictim's Sex: Female\nOffender's Sex: Male\nNumber of Victims: 1\nNumber of Offenders: 1", 'index': 0, 'logprobs': None, 'finish_reason': 'stop'}], 'usage': {'prompt_tokens': 1653, 'completion_tokens': 81, 'total_tokens': 1734}} ========
Inside try_api!!
======== Working on labeling case number 2 with GPT: 1264296 ========
======== Response.json(): {'id': 'cmpl-7NIdcsSRpSB2ngmBs63SSgR9EWsaw', 'object': 'text_completion', 'created': 1685787544, 'model': 'text-davinci-003', 'choices': [{'text': "\n\nYear: 2022\nDate: 14.11\nTime: Vormittags- und frühen Nachmittagsstunden\nType of Crime: 

======== Response.json(): {'id': 'cmpl-7NIehSroj6e8nQ7WTUWntn2x91ay2', 'object': 'text_completion', 'created': 1685787611, 'model': 'text-davinci-003', 'choices': [{'text': "\n\nYear: 2022\nDate: 10.11\nTime: 18.10 Uhr\nType of Crime: Homicide, Property Damage\nLocation: Seegefelder Straße, Spandau\nVictim's Sex: Male\nOffender's Sex: Unknown\nNumber of Victims: 1\nNumber of Offenders: 2", 'index': 0, 'logprobs': None, 'finish_reason': 'stop'}], 'usage': {'prompt_tokens': 1635, 'completion_tokens': 70, 'total_tokens': 1705}} ========
Inside try_api!!
======== Working on labeling case number 15 with GPT: 1263474 ========
======== Response.json(): {'id': 'cmpl-7NIeoWw5WMRslEYMnc2NFD5dpqyOL', 'object': 'text_completion', 'created': 1685787618, 'model': 'text-davinci-003', 'choices': [{'text': "\n\nYear: 2022\nDate: 16.11\nTime: 21 Uhr\nType of Crime: Property Damage\nLocation: Tempelhof-Schöneberg\nVictim's Sex: Male, Female\nOffender's Sex: Unknown\nNumber of Victims: 2\nNumber of Offend

In [452]:
info_dict = inject_cases_info_to_dict(original_dict_filled)

======== Extracting info from labeled case number 1: 1264683 ========
gpt_output_lst
['Year: 2022', 'Date: 15.11', 'Time: 11.45 Uhr', 'Type of Crime: Property Damage, General Assault', 'Location: Cantianstraße, Schönhauser Allee, Prenzlauer Berg, Pankow', "Victim's Sex: Female", "Offender's Sex: Male", 'Number of Victims: 1', 'Number of Offenders: 1']
======== Extracting info from labeled case number 2: 1264296 ========
gpt_output_lst
['Year: 2022', 'Date: 14.11', 'Time: Vormittags- und frühen Nachmittagsstunden', 'Type of Crime: Property Crime (Theft)', 'Location: Berlinweit', "Victim's Sex: Unknown", "Offender's Sex: Unknown", 'Number of Victims: Unknown', 'Number of Offenders: Unknown']
======== Extracting info from labeled case number 3: 1264284 ========
gpt_output_lst
['Year: 2022', 'Date: 14.11', 'Time: 20 Uhr', 'Type of Crime: General Assault, Property Crimes', 'Location: Prenzlauer Allee / Ostseestraße, Bornholmer Straße', "Victim's Sex: Male", "Offender's Sex: Male", 'Number o

In [453]:
df_final_sep = pd.DataFrame.from_dict(info_dict)
df_final_sep

,unique_case_id,official_case_id,type_of_crime,location,year,date,time,victim_sex,offender_sex,number_of_victims,number_of_offenders
0,000001_1264683,1264683,"Property Damage, General Assault","Cantianstraße, Schönhauser Allee, Prenzlauer B...",2022,15.11,11.45 Uhr,Female,Male,1,1
1,000002_1264296,1264296,Property Crime (Theft),Berlinweit,2022,14.11,Vormittags- und frühen Nachmittagsstunden,Unknown,Unknown,Unknown,Unknown
2,000003_1264284,1264284,"General Assault, Property Crimes","Prenzlauer Allee / Ostseestraße, Bornholmer St...",2022,14.11,20 Uhr,Male,Male,1,3
3,000004_1264123,1264123,Property Damage,Friedrichshain-Kreuzberg,2022,13.11,3 Uhr,"Male, Female",Male,4,2
4,000005_1264114,1264114,"Property Damage, General Assault, Verbal Abuse","Platz vor dem Neuen Tor, Invalidenstraße",2022,13.11,23 Uhr,"Male, Female",Male,3,2
5,000006_1264113,1264113,"Homicide, Property Damage","Charlottenstraße, Friedrichshain-Kreuzberg",2022,13.11,20.50 Uhr,Male,Male,1,1
6,000007_1264101,1264101,"Property Damage, Possession of Weapons","Cotheniusstraße, Prenzlauer Berg",2022,12.11,15.30 Uhr,Unknown,Male,Unknown,1
7,000008_1263966,1263966,"Property Crime, General Assault","Kaiser-Friedrich-Straße, Charlottenburg-Wilmer...",2022,03.05,Unknown,Unknown,Male,Unknown,3
8,000009_1263887,1263887,Property Crime,Gesundbrunnen,2022,11.11,Unknown,Unknown,Male,Unknown,1
9,000010_1263865,1263865,"Property Damage, Arson","Mittelbuschweg, Neukölln",2022,11.11,19.30 Uhr,Unknown,Unknown,0,Unknown


In [440]:
output_csv(path_to_json_files, df_final_sep)

file save with name: labeled_2_cases_2023-06-03 12-17.csv


#### Diagnosis

In [354]:
original_dict_filled

{}

In [355]:
info_dict

{'unique_case_id': [],
 'official_case_id': [],
 'type_of_crime': [],
 'location': [],
 'year': [],
 'date': [],
 'time': [],
 'victim_sex': [],
 'offender_sex': [],
 'number_of_victims': [],
 'number_of_offenders': []}

In [ ]:
#TODO:
# One hot encdoing
#Timestamp

In [ ]:
#Next Steps 
# Lena todo: make sure promt works in function for one case - (all cases)
# Hsin todo: loop through cases - output dataframe

## Backup